In [1]:
!kaggle datasets download -d crawford/emnist

Dataset URL: https://www.kaggle.com/datasets/crawford/emnist
License(s): CC0-1.0
 99%|██████████████████████████████████████▋| 1.23G/1.24G [00:07<00:00, 181MB/s]
100%|███████████████████████████████████████| 1.24G/1.24G [00:07<00:00, 178MB/s]


In [2]:
!pip install torch torchvision openvino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 MB 2.2 MB/s eta 0:00:000:00:0100:01


In [ ]:
!pip install openvino-dev[ONNX]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 87.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 85.8 MB/s eta 0:00:00:00:01


In [11]:
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import os
import zipfile
from openvino.runtime import Core
import numpy as np

# Ensure CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Unzip the EMNIST dataset
with zipfile.ZipFile('/kaggle/working/emnist.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 62)  # 62 classes in EMNIST balanced

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# Load and preprocess the EMNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.EMNIST(root='/kaggle/working/emnist', split='balanced', train=True, download=True, transform=transform)
test_dataset = datasets.EMNIST(root='/kaggle/working/emnist', split='balanced', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Testing function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

# Train on GPU
model_gpu = SimpleCNN().to(device)
optimizer_gpu = optim.Adam(model_gpu.parameters())

start_time = time.time()
for epoch in range(1, 3):  # Train for 2 epochs
    train(model_gpu, device, train_loader, optimizer_gpu, epoch)
gpu_train_time = time.time() - start_time

gpu_accuracy = test(model_gpu, device, test_loader)

# Convert to ONNX
dummy_input = torch.randn(1000,1,28,28).to(device)
torch.onnx.export(model_gpu, dummy_input, "emnist_model.onnx")

# Optimize with OpenVINO
core = Core()
ov_model = core.read_model("emnist_model.onnx")
compiled_model = core.compile_model(ov_model, "CPU")

# Inference function for OpenVINO model
def infer_openvino(model, data):
    # Reshape the input data to match the expected input shape of the model
    input_tensor = data.numpy()
    # Get the expected input shape from the model
    input_shape = model.input(0).get_shape()  
    # Reshape the input tensor to match the expected shape, padding with zeros if necessary
    input_tensor = np.pad(input_tensor, ((0, input_shape[0] - input_tensor.shape[0]), (0, 0), (0, 0), (0, 0)), 'constant')
    result = model(input_tensor)[0]
    return torch.from_numpy(result)[:data.shape[0]]  # Only take the relevant part of the output

# Test OpenVINO model
start_time = time.time()
correct = 0
for data, target in test_loader:
    output = infer_openvino(compiled_model, data)  # Call the modified infer_openvino function
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

cpu_inference_time = time.time() - start_time
cpu_accuracy = 100. * correct / len(test_dataset) # Divide by len(test_dataset) not len(test_loader.dataset)

# GPU Inference Time and FPS Measurement
gpu_inference_frames = 0
start_time = time.time()
with torch.no_grad():
    for data, target in test_loader:
        data = data.to(device)
        output = model_gpu(data)
        gpu_inference_frames += data.size(0)  # Increment frames processed by batch size
gpu_inference_time = time.time() - start_time
gpu_inference_fps = gpu_inference_frames / gpu_inference_time if gpu_inference_time > 0 else 0

# CPU (OpenVINO) Inference FPS Measurement
cpu_inference_frames = 0
start_time = time.time()
for data, target in test_loader:
    output = infer_openvino(compiled_model, data)  # Call the modified infer_openvino function
    cpu_inference_frames += data.size(0)  # Increment frames processed by batch size
cpu_inference_time = time.time() - start_time
cpu_inference_fps = cpu_inference_frames / cpu_inference_time if cpu_inference_time > 0 else 0


# Print the results
print(f"GPU Training Time: {gpu_train_time:.2f} seconds")
print(f"GPU Inference Accuracy: {gpu_accuracy:.2f}%")
print(f"GPU Inference Time: {gpu_inference_time:.2f} seconds")
print(f"GPU Inference FPS: {gpu_inference_fps:.2f}")
print(f"CPU (OpenVINO) Inference Time: {cpu_inference_time:.2f} seconds")
print(f"CPU (OpenVINO) Inference Accuracy: {cpu_accuracy:.2f}%")
print(f"CPU (OpenVINO) Inference FPS: {cpu_inference_fps:.2f}")


Train Epoch: 1 [0/112800 (0%)]	Loss: 4.135045
Train Epoch: 1 [6400/112800 (6%)]	Loss: 2.076803
Train Epoch: 1 [12800/112800 (11%)]	Loss: 1.492812
Train Epoch: 1 [19200/112800 (17%)]	Loss: 1.390569
Train Epoch: 1 [25600/112800 (23%)]	Loss: 1.153956
Train Epoch: 1 [32000/112800 (28%)]	Loss: 1.018777
Train Epoch: 1 [38400/112800 (34%)]	Loss: 0.965032
Train Epoch: 1 [44800/112800 (40%)]	Loss: 1.073152
Train Epoch: 1 [51200/112800 (45%)]	Loss: 0.812791
Train Epoch: 1 [57600/112800 (51%)]	Loss: 1.043101
Train Epoch: 1 [64000/112800 (57%)]	Loss: 0.961771
Train Epoch: 1 [70400/112800 (62%)]	Loss: 0.803099
Train Epoch: 1 [76800/112800 (68%)]	Loss: 0.749528
Train Epoch: 1 [83200/112800 (74%)]	Loss: 0.853523
Train Epoch: 1 [89600/112800 (79%)]	Loss: 0.990795
Train Epoch: 1 [96000/112800 (85%)]	Loss: 0.748865
Train Epoch: 1 [102400/112800 (91%)]	Loss: 0.733336
Train Epoch: 1 [108800/112800 (96%)]	Loss: 0.835138
Train Epoch: 2 [0/112800 (0%)]	Loss: 0.540039
Train Epoch: 2 [6400/112800 (6%)]	Loss: 0